In [1]:
import os
import re
from collections import defaultdict

import textract
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from pymorphy2 import MorphAnalyzer
from PyPDF2 import PdfReader, PdfWriter
from tqdm.autonotebook import tqdm
import pandas as pd

/tmp/ipykernel_5731/945261966.py:10: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
REPORTS_DIRECTORY = "reports"

RE_DERIVED = re.compile("\w+( -|- |-|! - )\w+")
RE_RUSSIAN_TEXT = re.compile("[а-яА-Я]+")
EXCLUDE_PATTERNS = re.compile("[A-Za-z0-9!#$%&'()*+,./:;<=>?@[\]^_`{|}~—\"\-]+")

stopwords_ru = stopwords.words("russian")

In [3]:
morph = MorphAnalyzer()
cp_names = {
    "ак",
    "акб",
    "акига",
    "алмазэргиэнбанк",
    "алрос",
    "альфабанк",
    "алюминиевый",
    "амвэя",
    "ангарский",
    "анк",
    "ао",
    "архангельский",
    "ас",
    "асэ",
    "атомпроект",
    "атомредметзолото",
    "атомэнергомаш",
    "атомэнергопроект",
    "атомэнергопром",
    "афк",
    "ашан",
    "аэрофлот",
    "балтика",
    "барс",
    "бат",
    "башнефть",
    "билайн",
    "бифид",
    "бф",
    "виктория",
    "владивостокский",
    "внипиэт",
    "волга",
    "волгателек",
    "волжский",
    "восток",
    "втб",
    "выксунский",
    "вымпелком",
    "вэб",
    "газпром",
    "газпромбанк",
    "гарантинвест",
    "генерировать",
    "геннадий",
    "гидропресс",
    "гк",
    "гмк",
    "гнц",
    "головной",
    "гольцблат",
    "грэс",
    "гтлк",
    "губкин",
    "дания",
    "дивизион",
    "евразийский",
    "еврохим",
    "елена",
    "енисейский",
    "еэс",
    "желдортранс",
    "зао",
    "зарубежнефть",
    "ик",
    "илим",
    "имииафрикантовый",
    "инжиниринговый",
    "инккапитал",
    "интер",
    "интернэшнл",
    "информация",
    "иркутск",
    "казаньоргсинтез",
    "казмунайгаз",
    "калининградский",
    "камаз",
    "касперский",
    "кб",
    "кемикалс",
    "ковровский",
    "комстаротс",
    "красцветмет",
    "крафтфудс",
    "кубань",
    "кузбассэнерго",
    "ленэнерго",
    "леруа",
    "лпк",
    "лср",
    "лукойл",
    "мвидео",
    "мдмбанк",
    "мегафон",
    "мерлен",
    "металлоинвест",
    "металлургический",
    "метафракс",
    "ммк",
    "монди",
    "мон’дэлиса",
    "моррис",
    "мосводоканал",
    "московский",
    "мрск",
    "мрф",
    "мсп",
    "мтс",
    "мхк",
    "неманский",
    "нестля",
    "ниаэп",
    "нижнекамскнефтехим",
    "нииар",
    "нипигаз",
    "ниу",
    "нк",
    "нлмк",
    "новатэк",
    "новогор",
    "новосибирский",
    "нордиск",
    "норильский",
    "оао",
    "объединение",
    "огк",
    "окб",
    "окбм",
    "около",
    "омк",
    "они",
    "ооо",
    "пао",
    "пейп",
    "пепеляев",
    "петропавловск",
    "пивоваренный",
    "пивоваров",
    "пигмент",
    "пик",
    "по",
    "полиметалл",
    "приволжье",
    "прикамье",
    "рао",
    "распадский",
    "ргу",
    "реновый",
    "ржд",
    "рксхолдинг",
    "рольф",
    "росатом",
    "росводоканал",
    "роснефть",
    "россеть",
    "российский",
    "россия",
    "ростелеком",
    "ростёха",
    "росэнергоатом",
    "рудна",
    "русало",
    "русгидро",
    "руссинвест",
    "руссия",
    "русый",
    "русь",
    "рф",
    "рязанский",
    "сабмиллер",
    "санофироссия",
    "сахалин",
    "сбербанк",
    "свез",
    "севергазпром",
    "севернефтегазпром",
    "северсталь",
    "сзлк",
    "сибирь",
    "сибурхолдинг",
    "система",
    "ситибанк",
    "ситроникс",
    "совкомбанк",
    "спбаэп",
    "суал",
    "сургутнефтегаз",
    "суэк",
    "схк",
    "сыктывкарский",
    "такед",
    "таманьнефтегаз",
    "татнефть",
    "твэл",
    "тгк",
    "техснабэкспорт",
    "тимченко",
    "тмк",
    "тнк",
    "тнквр",
    "тоаз",
    "трансаэро",
    "трансгаз",
    "транснефть",
    "тюмень",
    "урал",
    "уралкалий",
    "уралсиб",
    "уральский",
    "урсабанк",
    "ухта",
    "фгк",
    "ферреро",
    "фиабанк",
    "филиал",
    "филип",
    "фк",
    "фортум",
    "фосагро",
    "фпк",
    "фск",
    "химконцентрат",
    "центринвест",
    "черкизовый",
    "чусовской",
    "шелл",
    "шереметьево",
    "щуровский",
    "эксонмобила",
    "энела",
    "энергохолдинг",
    "энерджи",
    "эсэфай",
    "южнороссийский",
    "юкос",
    "юникредитбанк",
    "юнипро",
    "яндекс",
    "ятэк",
}
NOUNS = {"NOUN"}
VERBS = {"VERB", "INFN", "GRND"}
ADJ = {"ADJF", "ADJS", "COMP", "PRTF", "PRTS"}

In [4]:
info_df = pd.read_csv("rspp_reports.csv", index_col=0)
info_df.head()

,компания,сектор,год,тип отчета,ссылка на отчет
0,Центр Корпоративной Медицины,Здравоохранение и спорт,2022,ОУР,/download/b98f348b936bbc33387b248e52cb1f2a/
1,ДОМ.РФ,Финансы и страхование,2022,ОУР,/download/82e60c15b4c0ee6ede2e194aa31c69e1/
2,ПАО «Россети Ленэнерго»,Энергетика,2022,СО,/download/d8bc2c7e94424c65e37793a462b7807d1274...
3,"ОАО ""МРСК Урала""",Энергетика,2021,ИО,/download/af503e07dd6b861d1ed3048c36868cc9/
4,"ПАО ""Россети Сибирь""",Энергетика,2021,ИО,/download/57becde4be827f45bedf2a46f58d793a/


In [7]:
def parse_pages_to_texts():
    errors = []
    df = []
    for report_name in tqdm(os.listdir(REPORTS_DIRECTORY)):
        if not report_name.endswith(".pdf"):
            continue
        try:
            report = textract.process(f"{REPORTS_DIRECTORY}/{report_name}")
        except Exception:
            errors.append(report_name)
            continue
        company_name = report_name[:-4].split("_")[0]
        report_index = int(report_name[:-4].split("_")[-1])
        for i, paragraph in enumerate(report.decode().split("\n\n")):
            paragraph = paragraph.replace("\n", " ")
            report_page = RE_DERIVED.sub("", paragraph)
            report_page_lst = word_tokenize(report_page)
            # russian_report_page_lst = [w for w in filter(RE_RUSSIAN_TEXT.match, report_page_lst)]
            tokens = []
            nouns = set()
            verbs = set()
            adj = set()
            for word_ in filter(RE_RUSSIAN_TEXT.match, report_page_lst):
                if word_ and word_ not in stopwords_ru and word_ not in cp_names:
                    word_ = word_.strip()
                    word_ = morph.parse(word_)[0]
                    normal_form = word_.normal_form
                    tokens.append(normal_form)
                    # if word_.tag.POS in NOUNS:
                    #     nouns.add(normal_form)
                    # elif word_.tag.POS in VERBS:
                    #     verbs.add(normal_form)
                    # elif word_.tag.POS in ADJ:
                    #     adj.add(normal_form)
            # if len(tokens) == 0:
            #     continue
            df.append({
                "company": company_name,
                "year": info_df.at[report_index, "год"],
                "sector": info_df.at[report_index, "сектор"],
                "report_type": info_df.at[report_index, "тип отчета"],
                "paragraph": i,
                "original_text": paragraph,
                "cleaned_text": " ".join(tokens),
            })
    print(*errors, sep="\n")
    return df

In [8]:
df = parse_pages_to_texts()

  0%|          | 0/1399 [00:00<?, ?it/s]

ОАО «Рязанская ГРЭС»_1033.pdf
«Россети Северо-Запад»_977.pdf
ОАО «Фортум» (ОАО «ТГК-10»)_1350.pdf
ОАО «Рязанская ГРЭС»_1031.pdf
ОАО «Архангельский целлюлозно- бумажный комбинат» _208.pdf
ООО «НОВОГОР- Прикамье»_760.pdf
ОАО «Енисейская территориальная генерирующая компания» (ТГК-13)_443.pdf
ОАО «Енисейская территориальная генерирующая компания» (ТГК-13)_440.pdf
ПАО «Татнефть»_1252.pdf
ЗАО КБ «ФИА-БАНК»_1317.pdf
ОАО «Архангельский целлюлозно- бумажный комбинат» _206.pdf
ПАО «Уралкалий»_1276.pdf
ОАО «Фортум» (ОАО «ТГК-10»)_1349.pdf
ОАО «РЖД»_917.pdf
ПАО «ОГК-2»_796.pdf
ОАО «Архангельский целлюлозно- бумажный комбинат» _207.pdf
Объединение «Желдортранс»_834.pdf
ПАО «Уралкалий»_1277.pdf
Лаборатория Касперского_554.pdf
ПАО «ОГК-2»_795.pdf
ПАО «Газпром»_380.pdf
Владивостокский государственный университет экономики и сервиса_312.pdf
ПАО «Газпром»_379.pdf
ПАО «ВЫМПЕЛКОМ» _ БРЕНД БИЛАЙН_290.pdf
ПАО «ТОАЗ»_1223.pdf
ПАО «Газпром»_381.pdf
АО «ОКБМ им.И.И.Африкантова»_565.pdf
ОАО «ОГК-6»_802.pdf
ПАО

In [9]:
new_df = pd.DataFrame(df)
new_df.head()

,company,year,sector,report_type,paragraph,original_text,cleaned_text
0,ПАО «ММК»,2014,Металлургическая и горнодобывающая,СО,0,1 ММК,ммк
1,ПАО «ММК»,2014,Металлургическая и горнодобывающая,СО,1,СОЦИАЛЬНЫЙ ОТЧЕТ ММК ЗА 2014 ГОД МАГНИТОГОРСКИ...,социальный отчёт ммк за год магнитогорский мет...
2,ПАО «ММК»,2014,Металлургическая и горнодобывающая,СО,2,www.mmk.ru,
3,ПАО «ММК»,2014,Металлургическая и горнодобывающая,СО,3,СОДЕРЖАНИЕ,содержание
4,ПАО «ММК»,2014,Металлургическая и горнодобывающая,СО,4,Обращение председателя совета директоров.........,обращение председатель совет директор параметр...


In [10]:
new_df.to_csv("paragraphs.csv.zip")